# Text Generation with RNNs


In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from collections import OrderedDict



### 1 Dataset
Define the path of the file, you want to read and train the model on


In [2]:
# link given in https://discord.com/channels/769888668915597342/808396835051536424/812083278730362930
![ -e '2701.txt' ] || wget 'https://raw.githubusercontent.com/GITenberg/Moby-Dick--Or-The-Whale_2701/master/2701.txt'

--2021-02-22 08:05:15--  https://raw.githubusercontent.com/GITenberg/Moby-Dick--Or-The-Whale_2701/master/2701.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1257304 (1.2M) [text/plain]
Saving to: ‘2701.txt’

2701.txt            100%[===================>]   1.20M  --.-KB/s    in 0.02s   

2021-02-22 08:05:16 (49.3 MB/s) - ‘2701.txt’ saved [1257304/1257304]



In [3]:
path_to_file = './2701.txt'
text = open(path_to_file, encoding='utf-8').read().replace('\r\n', '\n')

In [4]:
text = text[23047:]  # seek to chapter 1
text = text[:100000]


#### Inspect the dataset
Take a look at the first 250 characters in text

In [5]:
print(text[:250])



CHAPTER 1. Loomings.


Call me Ishmael. Some years ago--never mind how long precisely--having
little or no money in my purse, and nothing particular to interest me on
shore, I thought I would sail about a little and see the watery part of
the world. 


In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


73 unique characters


### 2 Process the dataset for the learning task
The task that we want our model to achieve is: given a character, or a sequence of characters, what is the most probable next character?

To achieve this, we will input a sequence of characters to the model, and train the model to predict the output, that is, the following character at each time step. RNNs maintain an internal state that depends on previously seen elements, so information about all characters seen up until a given moment will be taken into account in generating the prediction.

#### Vectorize the text
Before we begin training our RNN model, we'll need to create a numerical representation of our text-based dataset. To do this, we'll generate two lookup tables: one that maps characters to numbers, and a second that maps numbers back to characters. Recall that we just identified the unique characters present in the text.

In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
text_as_int = np.array([char2idx[c] for c in text])

# Create a mapping from indices to characters
idx2char = np.array(vocab)


This gives us an integer representation for each character. Observe that the unique characters (i.e., our vocabulary) in the text are mapped as indices from 0 to len(unique). Let's take a peek at this numerical representation of our dataset:

In [8]:

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '1' :  10,
  '2' :  11,
  '3' :  12,
  '4' :  13,
  '5' :  14,
  '6' :  15,
  '7' :  16,
  '8' :  17,
  '9' :  18,
  ':' :  19,
  ...
}




We can also look at how the first part of the text is mapped to an integer representation:

In [9]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))



'CHAPTER 1. Lo' ---- characters mapped to int ---- > [24 29 22 37 41 26 39  1 10  9  1 33 61]


#### Defining a method to encode one hot labels

In [10]:
def one_hot_encode(arr, n_labels):
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot


#### Defining a method to make mini-batches for training

In [11]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''

    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr) // batch_size_total

    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))

    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n + seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n + seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y




## 3 The Recurrent Neural Network (RNN) model


###### Check if GPU is available

In [12]:
train_on_gpu = torch.cuda.is_available()
print ('Training on GPU' if train_on_gpu else 'Training on CPU')

Training on GPU



### Declaring the model

In [13]:
class VanillaCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        n_features = len(vocab) + n_hidden
        self.body = nn.Sequential(OrderedDict([
                                   layer
                                   for i in range(n_layers)
                                   for layer in [
                                     (f'linear_{i}', nn.Linear(n_features, n_features)),
                                     (f'tanh', nn.Tanh()) if i == n_layers - 1 else (f'relu_{i}', nn.ReLU())
        ]]))


    def forward(self, x, hidden):

        n_hidden = self.n_hidden
        if type(hidden) == tuple:
          hidden = torch.vstack(hidden)
        y = torch.zeros_like(x)
        hidden_t = hidden
        for i in range(x.shape[1]):
          xi = x[:, i]
          yi = self.body(torch.hstack([xi, hidden_t]))
          yi, hidden_t = yi[:, :-n_hidden], yi[:, -n_hidden:]
          y[:, i] = yi

        # return the final output and the hidden state
        return y, hidden_t

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        
        hidden = torch.zeros((batch_size, self.n_hidden))
        if train_on_gpu:
          hidden = hidden.cuda()

        return hidden

In [14]:
class LSTMCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        n_features = len(vocab) + n_hidden
        self.block_c = nn.Sequential(OrderedDict([
              ('linear', nn.Linear(n_features, n_features)),
              ('tanh', nn.Tanh())
        ]))
        self.block_f = nn.Sequential(OrderedDict([
              ('linear', nn.Linear(n_features, n_features)),
              ('sigmoid', nn.Sigmoid())
        ]))
        self.block_i = nn.Sequential(OrderedDict([
              ('linear', nn.Linear(n_features, n_features)),
              ('sigmoid', nn.Sigmoid())
        ]))
        self.block_o = nn.Sequential(OrderedDict([
              ('linear', nn.Linear(n_features, n_features)),
              ('sigmoid', nn.Sigmoid())
        ]))


    def forward(self, x, hidden):
        # return the final output and the hidden state
        n_hidden = self.n_hidden
        y = torch.zeros_like(x)
        hidden_t, C = hidden
        for i in range(x.shape[1]):
          xi = x[:, i]
          yi, C = self.forward_one(torch.hstack([xi, hidden_t]), C)
          yi, hidden_t = yi[:, :-n_hidden], yi[:, -n_hidden:]
          y[:, i] = yi
        return y, (hidden_t, C)
    
    def forward_one(self, xh, C):
        simC = self.block_c(xh)
        f = self.block_f(xh)
        i = self.block_i(xh)
        Cnew = f * C + i * simC
        o = self.block_o(xh)
        yh = o * F.tanh(Cnew)
        return yh, Cnew

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        
        hidden = torch.zeros((batch_size, self.n_hidden))
        C = torch.zeros((batch_size, self.n_hidden + len(self.vocab)))
        if train_on_gpu:
          hidden = hidden.cuda()
          C = C.cuda()

        return hidden, C


#### Declaring the train method

In [15]:
def train(model, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        model: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
    model.train()

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if (train_on_gpu):
        model.cuda()

    counter = 0
    n_vocab = len(model.vocab)
    for e in range(epochs):
        # initialize hidden state
        h = model.init_hidden(batch_size)
        
        dataset = get_batches(data, batch_size, seq_length)

        for x, y in dataset:
            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_vocab)
            y = y.reshape(-1)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y).long()

            if (train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            model.zero_grad()
            
            '''TODO: feed the current input into the model and generate output'''
            output, h = model(inputs, h)

            loss = criterion(output.reshape((len(targets), -1)), targets)
            
            # perform backprop
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = model.init_hidden(batch_size)
                val_losses = []
                model.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_vocab)
                    y = y.reshape(-1)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if (train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = model(inputs, val_h)
                    
                    val_loss = criterion(output.reshape((len(targets), -1)), targets)

                    val_losses.append(val_loss.item())

                print("Epoch: {}/{}...".format(e + 1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
                input_eval = "The "
                print(sample(model, 1000, prime=input_eval, top_k=5))
                
                model.train()  # reset to train mode after iterationg through validation data


##### Defining a method to generate the next character

In [16]:
def predict(model, char, h=None, top_k=None):
    ''' Given a character, predict the next character.
        Returns the predicted character and the hidden state.
    '''

    # tensor inputs
    x = np.array([[char2idx[char]]])
    x = one_hot_encode(x, len(model.vocab))
    inputs = torch.from_numpy(x)

    if (train_on_gpu):
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    out, h = model(inputs, h)

    # get the character probabilities
    p = F.softmax(out, dim=1).data
    if (train_on_gpu):
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(model.vocab))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return idx2char[char], h



#### Declaring a method to generate new text

In [17]:
def sample(model, size, prime='The', top_k=None):
    if (train_on_gpu):
        model.cuda()
    else:
        model.cpu()

    model.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1)
    for ch in prime:
        char, h = predict(model, ch, h, top_k=top_k)

    chars.append(char)

    for ii in range(size):
      '''TODO: pass in the previous character and get a new one'''
      char, h = predict(model, ch, h, top_k=top_k)
      chars.append(char)

    model.train()
    return ''.join(chars)


#### Generate new Text using the RNN model

###### Define and print the net

In [18]:

n_hidden = 512
n_layers = 1

In [19]:
vanilla_model = VanillaCharRNN(vocab, n_hidden, n_layers)
print(vanilla_model)


VanillaCharRNN(
  (body): Sequential(
    (linear_0): Linear(in_features=585, out_features=585, bias=True)
    (tanh): Tanh()
  )
)


In [20]:

lstm_model = LSTMCharRNN(vocab, n_hidden, n_layers)
print(lstm_model)

LSTMCharRNN(
  (block_c): Sequential(
    (linear): Linear(in_features=585, out_features=585, bias=True)
    (tanh): Tanh()
  )
  (block_f): Sequential(
    (linear): Linear(in_features=585, out_features=585, bias=True)
    (sigmoid): Sigmoid()
  )
  (block_i): Sequential(
    (linear): Linear(in_features=585, out_features=585, bias=True)
    (sigmoid): Sigmoid()
  )
  (block_o): Sequential(
    (linear): Linear(in_features=585, out_features=585, bias=True)
    (sigmoid): Sigmoid()
  )
)



###### Declaring the hyperparameters

In [21]:
batch_size = 20
seq_length = 100
n_epochs = 80


##### Train the model and have fun with the generated texts

In [22]:
torch.random.manual_seed(42)

In [23]:
train(vanilla_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=100)

Epoch: 3/80... Step: 100... Loss: 3.3774... Val Loss: 3.3813
The dcdcZdccbaaZaZdaZcZcddbZbbaaaZZZZabbacbZZadaZZcZcaZcadZdZdaaabcaadbadZdcccdaaacZdaabacadbaabaZacacdabbcaabaZcZaacdbbdbZaacaaZbdabZbcbZccaacZabbcaccccbZbadddbbZZcbZbbcadddbbadadbadbdccccbbbZdbaZabaadcbacadacZcdbcadbbdcdbddabZdaddddcabZcZddccddcbabcbcbdbcccadcbabcadcbdbdadaZccdcaZbdcdaZaZdaacbdaaZbccdcZZZbZbadaZcZbZaccccZbcaZbZbcbdZadZcZZZbdbadccbadcddbcbaccbdbZcdbcccbddbdadaZaababbZcZcbZdadddcbbccbadbbaZbcbcccbZZaZbcdcZccdabZddccZbdaZadccabdabZZZabdcddcbbddbddZbZdbccbccbcbdaZbcdbcbccaccZaadZcaZbacdZcadZZaZbZbbZZcZZbZcaZacZabZcccbaZdbcddbadacbdZcaZZZZZbaaZdbcdbbcbdbddZaaaZccZddddbdZcccaddadddbdaZcaZcdZdadZbZabZbbdbbZdcacdbdddabacaacdZaZZaZbbdcddZZcdacdcbdcccZcdZbdcaZaaaaabbbaaacdZaaddccdZcaZdcZcdbaZaZbZcaaZadccZabcccZccZcZZZZZbZcdbcZaaZcZZZdbdbccaZacZZbcaacbcbZaZdZZdcdcccZZaaaaZaacacbbbbbcabZbZddccddZbdbZbaZadcabbcbddZdacadaZZdbabcZZcaaZddZcbZcZaZbZaZZacabdcbdadddacdbcbdaaacabZZbcdccabbcddcZcZdadadaaacZZZZacabcddbZacacaccac

In [24]:
train(lstm_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1628: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch: 2/80... Step: 50... Loss: 3.4129... Val Loss: 3.3970
The aZcabcbacaaZddZZcbZabaZabddabaaZdcaaaZcaaZcZaaZacabZZdaaZabdccaZcabadcadZabcbZdcbZdZdZbaabZabcdacbacccbdaacdZdZbabaZZdccccZcbabdadbbdZZadcZZccdcbbaddbdababZcbZZbdcdacbbcbbccZcbddbabcdbcZadcaaZaZbcdbZaaZcbacZbZbbcdZZbZdZZdddccaZdaacZZadbZZdcdcbZbddddacZcdaZcZabddbbdZdabZcacaZadZZabbZadadadbdZabbbdaZcZZbZcZdcbbddbbaZbcZZdaabbabZabddadZcdacadbabaacZbcddaaabdcZdZaccdadZbcZaddZdaZbaabZabbacdacdbadZaacaaZdbbdccdZbaccabcaZdababbZbcZbcZZacZcZadZadaabZdcbbacdbZddZdZacaZbccbaaaddaZbcdbZddacdccZbddabbdbdbdcddZcabdcbaZZbcadcabadacddZdZaZccdbadZZcdccdbdadbcaZZZZZdcZddbZZaZZZcbbbdbZcddabbbccddZbcbZacacbaZcdacccZZZddacZbccZZccbdZZZbbdabZaZabdddaZdZcdZbddbcbbccZcZdcaccdbZcZddZbbdZddbZZZcdccabadcccdbbabbdZabZddZdaZZdcacZaZbbbbacdcbZZbbaZdcbdabaabbaddbbZZZbZabZdbbcacZcbcZdZZddddccacaacbcabdaZcdZcddadcZddacaddbZbadbddacbdZaZdbZdcbabZbaddZdcaZZdacbZccdZabaZZbaZdZbbdbcaZZZdZZbcbZacaadbddcbdZZaZZcdbZbZZabccbcZbaccbccaacbbbdadaZccabZbaccZdcbddZc

Both model's didn't give any reasonable output, vanilla RNN had smaller validation loss.